## Designing a Neural Network 

In [3]:
# Import dependencies
import torch
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Get data
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, batch_size=32, shuffle=True)

In [6]:
# Image Classifier Neural Network
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)
        )

    def forward(self, x):
        return self.model(x)

# Instance of the neural network, loss, optimizer
clf = ImageClassifier().to('cpu')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training flow
for epoch in range(10):  # train for 10 epochs
    for X, y in dataset:
        X, y = X.to('cpu'), y.to('cpu')
        yhat = clf(X)
        loss = loss_fn(yhat, y)

        # Apply backprop
        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"Epoch:{epoch} loss is {loss.item()}")

# Save the model state
torch.save(clf.state_dict(), 'model_state.pt')


Epoch:0 loss is 0.03428696468472481
Epoch:1 loss is 0.009311122819781303
Epoch:2 loss is 0.005131776910275221
Epoch:3 loss is 0.002006225287914276
Epoch:4 loss is 0.002772724023088813
Epoch:5 loss is 0.008340626023709774
Epoch:6 loss is 0.00915125198662281
Epoch:7 loss is 1.239264656760497e-05
Epoch:8 loss is 8.039805834414437e-05
Epoch:9 loss is 1.761588100634981e-05


# Prediction

In [7]:
import torch
from torchvision.transforms import ToTensor
from PIL import Image

clf = ImageClassifier()

# Load Model State Dict
model_path = 'model_state.pt'
with open(model_path, 'rb') as f:
    clf.load_state_dict(torch.load(f))
    clf.eval()

In [8]:
# Pre-Process the Image for Prediction
img_path = 'img_1.jpg'
img = Image.open(img_path).convert('L')

transform = ToTensor()

img_tensor = transform(img).unsqueeze(0).to('cpu')

In [10]:
with torch.no_grad():
    output = clf(img_tensor)

prediction = torch.argmax(output).item()
print("Predicted Digit: ", prediction)

Predicted Digit:  2


#### ~~~~~~ SUCCESS ~~~~~~~~~ ####